In [ ]:
#######################################################
#######################################################
############    COPYRIGHT - DATA SOCIETY   ############
#######################################################
#######################################################

## INTRO TO TEXT MINING PART 2/INTRO TO TEXT MINING PART 2 ##

## NOTE: To run individual pieces of code, select the line of code and
##       press ctrl + enter for PCs or command + enter for Macs




In [ ]:
#=================================================-
#### Slide 6: Loading packages  ####

# Helper packages.
from pathlib import Path
import os
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
# Packages with tools for text processing.
import nltk
import nltk.data
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

# We will introduce these packages today.
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer




In [ ]:
#=================================================-
#### Slide 7: Loading packages (cont'd)  ####

# Packages for getting data ready for and building a LDA model.
import gensim
from gensim import corpora, models
from pprint import pprint




In [ ]:
#=================================================-
#### Slide 8: Directory settings  ####

# Set 'main_dir' to location of the project folder
home_dir = Path(".").resolve()
main_dir = home_dir.parent
print(main_dir)
data_dir = str(main_dir) + "/data"
print(data_dir)




In [ ]:
#=================================================-
#### Slide 9: Loading pickles  ####

NYT_tokenized = pickle.load(open(str(data_dir) + "/NYT_tokenized.sav","rb"))  
word_counts_array = pickle.load(open(str(data_dir) + "/word_counts_array.sav","rb"))  
processed_docs = pickle.load(open(str(data_dir) + "/NYT_clean.sav","rb"))  
NYT_clean_list = pickle.load(open(str(data_dir) + "/NYT_clean_list.sav","rb"))  




In [ ]:
#=================================================-
#### Slide 13: Create a DTM  ####

# Initialize `CountVectorizer`.
vec = CountVectorizer()

# Transform the list of snippets into DTM.
X = vec.fit_transform(NYT_clean_list)
print(X.toarray()) #<- show output as a matrix

print(vec.get_feature_names()[:10])




In [ ]:
#=================================================-
#### Slide 14: Create a DTM (cont'd)  ####

# Convert the matrix into a pandas dataframe for easier manipulation.
DTM = pd.DataFrame(X.toarray(), columns = vec.get_feature_names())
print(DTM.head())




In [ ]:
#=================================================-
#### Slide 16: DTM to dictionary of total word counts (cont'd)  ####

# Create a convenience function that sorts and looks at first n-entries in the dictionary.
def HeadDict(dict_x, n):
    # Get items from the dictionary and sort them by
    # value key in descending (i.e. reverse) order
    sorted_x = sorted(dict_x.items(), 
                      reverse = True, 
                      key = lambda kv: kv[1])
                      
    # Convert sorted dictionary to a list.
    dict_x_list = list(sorted_x)
    
    # Return the first `n` values from the dictionary only.
    return(dict(dict_x_list[:n]))




In [ ]:
#=================================================-
#### Slide 17: DTM to dictionary of total word counts (cont'd)  ####

# Sum frequencies of each word in all documents.
DTM.sum(axis = 0).head()

# Save series as a dictionary.
corpus_freq_dist = DTM.sum(axis = 0).to_dict()

# Glance at the frequencies.
print(HeadDict(corpus_freq_dist, 6))




In [ ]:
#=================================================-
#### Slide 19: Plot distribution of words in snippet corpus  ####

# Save as a FreqDist object native to nltk.
corpus_freq_dist = nltk.FreqDist(corpus_freq_dist)
# Plot distribution for the entire corpus.
plt.figure(figsize = (16, 7))
corpus_freq_dist.plot(80)




In [ ]:
#=================================================-
#### Slide 21: Visualizing word counts with word clouds  ####


# Construct a word cloud from corpus.
wordcloud = WordCloud(collocations = False, max_font_size = 40, background_color = "white")
wordcloud = wordcloud.generate(' '.join(NYT_clean_list))

# Plot the cloud using matplotlib.
plt.figure()
plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis("off")
plt.show()




In [ ]:
#=================================================-
#### Slide 25: Exercise 1  ####






In [ ]:
#=================================================-
#### Slide 50: Create a dictionary of counts   ####

# Set the seed.
np.random.seed(1)

dictionary = gensim.corpora.Dictionary(processed_docs)

# The loop below iterates through the first 10 items of the dictionary and prints out the key and value. 
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break




In [ ]:
#=================================================-
#### Slide 51: Create a dictionary of counts   ####

dictionary.filter_extremes(no_below = 5, no_above = 0.5, keep_n = 248)

# How many words did are left in the dictionary?
len(dictionary)




In [ ]:
#=================================================-
#### Slide 52: Document to bag-of-words  ####

# We use a list comprehension to transform each doc within our processed_docs object.
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Let's look at the first document.
print(bow_corpus[0])




In [ ]:
#=================================================-
#### Slide 53: Document to bag-of-words  ####

# Isolate the first document.
bow_doc_1 = bow_corpus[0]

# Iterate through each dictionary item using the index. 
# Print out each actual word and how many times it appears.
for i in range(len(bow_doc_1)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_1[i][0],
          
          dictionary[bow_doc_1[i][0]],
          bow_doc_1[i][1]))




In [ ]:
#=================================================-
#### Slide 56: Transform counts with TfidfModel  ####

# This is the transformation.
tfidf = models.TfidfModel(bow_corpus)

# Apply the transformation to the entire corpus.
corpus_tfidf = tfidf[bow_corpus]

# Preview TF-IDF scores for the first document.
for doc in corpus_tfidf:
    pprint(doc)
    break




In [ ]:
#=================================================-
#### Slide 58: Save results as a pickle  ####

pickle.dump(bow_corpus, open(str(data_dir) +'/bow_corpus.sav', 'wb'))
pickle.dump(corpus_tfidf, open(str(data_dir) + '/corpus_tfidf.sav', 'wb'))
pickle.dump(dictionary, open(str(data_dir) + '/dictionary.sav', 'wb'))
pickle.dump(DTM, open(str(data_dir) + '/DTM.sav', 'wb'))
pickle.dump(X, open(str(data_dir) + '/DTM_matrix.sav', 'wb'))


